# Part-of-speech tagging

### Import libraries

In [47]:
import os


### Download dataste

In [48]:
import urllib.request  #  download files
import zipfile  #  unzip files

dataset_folder = os.path.join(os.getcwd(), "OriginalDataset")

if not os.path.exists(dataset_folder):
    os.makedirs(dataset_folder)

url = 'https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/packages/corpora/dependency_treebank.zip'

dataset_path = os.path.join(dataset_folder, "dependency_treebank.zip")

if not os.path.exists(dataset_path):
    urllib.request.urlretrieve(url, dataset_path)
    print("Successful download")

with zipfile.ZipFile(dataset_path,"r") as zip_ref:
    zip_ref.extractall(dataset_folder)
print("Successful extraction")

Successful extraction


### Split dataset

In [49]:
train_path = './Dataset/train/'# Should we use: os.path.join(os.getcwd(), "OriginalDataset")
val_path = './Dataset/val/'
test_path = './Dataset/test/'
original_path = dataset_folder +'/dependency_treebank/'

In [50]:
if not os.path.exists(train_path):
    print("making directory")
    os.makedirs(train_path)
if not os.path.exists(val_path):
    os.makedirs(val_path)
if not os.path.exists(test_path):
    os.makedirs(test_path)

making directory


In [51]:
original_dataset = os.listdir(original_path)
print(original_dataset)

['wsj_0095.dp', 'wsj_0184.dp', 'wsj_0177.dp', 'wsj_0037.dp', 'wsj_0126.dp', 'wsj_0066.dp', 'wsj_0052.dp', 'wsj_0112.dp', 'wsj_0003.dp', 'wsj_0143.dp', 'wsj_0153.dp', 'wsj_0013.dp', 'wsj_0102.dp', 'wsj_0042.dp', 'wsj_0076.dp', 'wsj_0136.dp', 'wsj_0027.dp', 'wsj_0167.dp', 'wsj_0194.dp', 'wsj_0085.dp', 'wsj_0007.dp', 'wsj_0147.dp', 'wsj_0056.dp', 'wsj_0116.dp', 'wsj_0180.dp', 'wsj_0091.dp', 'wsj_0122.dp', 'wsj_0062.dp', 'wsj_0173.dp', 'wsj_0033.dp', 'wsj_0023.dp', 'wsj_0163.dp', 'wsj_0072.dp', 'wsj_0132.dp', 'wsj_0081.dp', 'wsj_0190.dp', 'wsj_0106.dp', 'wsj_0046.dp', 'wsj_0157.dp', 'wsj_0017.dp', 'wsj_0006.dp', 'wsj_0146.dp', 'wsj_0057.dp', 'wsj_0117.dp', 'wsj_0181.dp', 'wsj_0090.dp', 'wsj_0123.dp', 'wsj_0063.dp', 'wsj_0172.dp', 'wsj_0032.dp', 'wsj_0022.dp', 'wsj_0162.dp', 'wsj_0073.dp', 'wsj_0133.dp', 'wsj_0080.dp', 'wsj_0191.dp', 'wsj_0107.dp', 'wsj_0047.dp', 'wsj_0156.dp', 'wsj_0016.dp', 'wsj_0094.dp', 'wsj_0185.dp', 'wsj_0176.dp', 'wsj_0036.dp', 'wsj_0127.dp', 'wsj_0067.dp', 'wsj_0053

In [52]:
original_dataset.sort()
org_train = original_dataset[0:100]
org_val = original_dataset[100:150]
org_test = original_dataset[150:]

print(org_train)
print(org_val)
print(org_test)


['wsj_0001.dp', 'wsj_0002.dp', 'wsj_0003.dp', 'wsj_0004.dp', 'wsj_0005.dp', 'wsj_0006.dp', 'wsj_0007.dp', 'wsj_0008.dp', 'wsj_0009.dp', 'wsj_0010.dp', 'wsj_0011.dp', 'wsj_0012.dp', 'wsj_0013.dp', 'wsj_0014.dp', 'wsj_0015.dp', 'wsj_0016.dp', 'wsj_0017.dp', 'wsj_0018.dp', 'wsj_0019.dp', 'wsj_0020.dp', 'wsj_0021.dp', 'wsj_0022.dp', 'wsj_0023.dp', 'wsj_0024.dp', 'wsj_0025.dp', 'wsj_0026.dp', 'wsj_0027.dp', 'wsj_0028.dp', 'wsj_0029.dp', 'wsj_0030.dp', 'wsj_0031.dp', 'wsj_0032.dp', 'wsj_0033.dp', 'wsj_0034.dp', 'wsj_0035.dp', 'wsj_0036.dp', 'wsj_0037.dp', 'wsj_0038.dp', 'wsj_0039.dp', 'wsj_0040.dp', 'wsj_0041.dp', 'wsj_0042.dp', 'wsj_0043.dp', 'wsj_0044.dp', 'wsj_0045.dp', 'wsj_0046.dp', 'wsj_0047.dp', 'wsj_0048.dp', 'wsj_0049.dp', 'wsj_0050.dp', 'wsj_0051.dp', 'wsj_0052.dp', 'wsj_0053.dp', 'wsj_0054.dp', 'wsj_0055.dp', 'wsj_0056.dp', 'wsj_0057.dp', 'wsj_0058.dp', 'wsj_0059.dp', 'wsj_0060.dp', 'wsj_0061.dp', 'wsj_0062.dp', 'wsj_0063.dp', 'wsj_0064.dp', 'wsj_0065.dp', 'wsj_0066.dp', 'wsj_0067

In [53]:
for f in org_train:
    os.rename(original_path+f, train_path+f)
for f in org_val:
    os.rename(original_path+f, val_path+f)
for f in org_test:
    os.rename(original_path+f, test_path+f)
